In [ ]:
import matplotlib
matplotlib.rc("text", usetex=True)
matplotlib.rc("font", family="serif")
import matplotlib.pyplot as plt
import matplotlib.colors as colors

import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
import h5py
import os
import sys
sys.path.append('.')
from datalib import Data, flag_to_species
from scipy.ndimage import gaussian_filter1d

In [ ]:
# This loads the data from the target directory into python
data = Data('../problems/two_stream/bin/Data/')
print(data.conf)
print(data.fld_steps[-1])

In [ ]:
# We would like to plot the electron phase space distribution of the last output timestep
data.reload()
step = data.ptc_steps[-1]
# step = 40
data.load(step)
print(data.time)

tick_size = 16
label_size = 24

fig, ax = plt.subplots(1, 1)
fig.set_size_inches(10.0, 6.0)
fig.patch.set_facecolor('w')

print(data.tracked_ptc_p1.shape)
e_x1 = data.tracked_ptc_x1[flag_to_species(data.tracked_ptc_flag) == 0]
e_p1 = data.tracked_ptc_p1[flag_to_species(data.tracked_ptc_flag) == 0]
p_x1 = data.tracked_ptc_x1[flag_to_species(data.tracked_ptc_flag) == 1]
p_p1 = data.tracked_ptc_p1[flag_to_species(data.tracked_ptc_flag) == 1]
print(e_x1)
ax.hist2d(e_x1, e_p1, bins=200)

ax.tick_params(labelsize=tick_size)
ax.set_xlabel(r'$x$', fontsize=label_size)
ax.set_ylabel(r'$p_x$', fontsize=label_size)
ax.text(0.7, 1.05, f'$t = {data.time:.2f}$',transform=ax.transAxes,
        fontsize=label_size)

In [ ]:
# This loop plots electron phase space for all output time steps, and save the plots in the "python/plots/" directory
if not os.path.exists("plots"):
    os.makedirs("plots")

tick_size = 16
label_size = 24
data.reload()
for step in data.fld_steps:
    fig, ax = plt.subplots(1, 1)
    fig.set_size_inches(10.0, 6.0)
    fig.patch.set_facecolor('w')

    data.load(step)
    
    e_x1 = data.tracked_ptc_x1[flag_to_species(data.tracked_ptc_flag) == 0]
    e_p1 = data.tracked_ptc_p1[flag_to_species(data.tracked_ptc_flag) == 0]
    ax.hist2d(e_x1, e_p1, bins=200, rasterized=True)

    ax.tick_params(labelsize=tick_size)
    ax.set_xlabel(r'$x$', fontsize=label_size)
    ax.set_ylabel(r'$p_x$', fontsize=label_size)
    
    ax.text(0.7, 1.05, f'$t = {data.time:.2f}$',transform=ax.transAxes,
            fontsize=label_size)
    fig.savefig(f'plots/{step:05d}.png')
    plt.close(fig)

In [ ]:
# growth rate
data.reload()
UE = np.zeros(len(data.fld_steps))
ts = np.zeros_like(UE)
gamma = np.sqrt(1.0 + data.conf['p0']**2)
print(gamma)
km = np.sqrt(3.0/8.0/gamma)/data.conf['p0']
print("km is ", km)
omega_p = np.sqrt(data.conf["rho_b"])
print("omega_p is", omega_p)
for step in data.fld_steps:
    data.load_fld(step)
    UE[step] = np.sum(np.sqrt(data.E1**2))
    ts[step] = data.time * omega_p
    print(step, ts[step], UE[step])

In [ ]:
tick_size = 25
label_size = 30

fig, ax = plt.subplots(1, 1)
fig.set_size_inches(10.0, 6.0)
fig.patch.set_facecolor('w')

ax.plot(ts, UE)
ax.set_yscale('log')
# ax.set_xlim(1,20)
# ax.set_ylim(1e-1, 1e3)

rate = 1.0 / (2.0 * np.sqrt(gamma**3))
print(rate, np.sqrt(1.0 + data.conf['p0']**2))
ys = 4.5e-2 * np.exp((ts-0) * rate)
ax.plot(ts, ys, '--')
print(data.conf['p0'])